<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C3-Owning_The_Network_With_Scapy/Processing_pcap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Processing pcap__*

Wireshark and other tools like Network Miner are great for interactively exploring packet capture files, but at times you'll want to slice and dice pcap files using Python and Scapy. Some great use cases are generating fuzzing test cases based on captured network traffic or even something as simple as replaying traffic that you have previously captured.

We'll take a slightly different spin on this and attempt to carve out image files from HTTP traffic. With these image files in hand, we will use OpenCV (http://www.opencv.org/), computer vision tool, to attempt to detect images that contain human faces so that we can narrow down images that might be interesting. You can use the previous ARP poisoning scirpt to generate the pcap files, or you could extend the ARP poisoning sniffer to do on-the-fly facial detection of images while the target is browsing.

This example will perform two separate tasks: Carving images out of HTTP traffic and detecting faces in those images. To accommodate this, we'll create two programs so that you can choose to use them separately, depending on the task at hand. You could also use the programs in sequence, as we'll do here. The first program, __recapper.py__, analyzes a pcap file, locates any images that are present in the streams contained in the pcap file, and writes those images to disk. The second program, __detector.py__, analyzes each of those image files to determine if it contains a face. If it does, it writes a new image to disk, adding a box around each face in the image.

Let's get started by dropping in the code necessary to perform the pcap analysis. In the following code, we'll use a __namedtuple__, a Python data structure with fields accessible by attribute lookup. A standard tuple enables you to store a sequence of immutable values; they're almost like lists, except you can't change a tuple's value. The standard tuple uses numerical indexes to access its members
```
point = (1.1, 2.5)
print(point[0], point[1])
```
A __namedtuple__, on the other hand, behaves the same as a regular tuple except that it can access fields through their names. This makes for much more readable code and is also more memory-efficient than a dictionary. The syntax to create a __namedtuple__ requires two arguments: The tuple's name and a space-separated list of field names. For example, say you want to create a data structure called __Point__ with two attributes: __x__ and __y__. You'd define it as follows:
```
Point = namedtuple("Point", ['x', 'y'])
```
Then you could create a __Point__ object named __p__ for example, with the code:
```
p = Point(35, 65)
```
And refer to its attributes just like those of a class: __p.x__ and __p.y__ refer to the __x__ and __y__ attributes of a particular __Point namedtuple__. That is much easier to read than code referring to the index of some item in a regular tuple. In our example, say you create a __namedtuple__ called __Response__ with the following code:
```
Response = namedtuple("Response", ["header", "payload"])
```
Now, instead of referring to an index of a normal tuple, you can use __Response.header__ or __Response.payload__, which is much easier to understand.

Let's use that information in this example. We'll read a __pcap__ file, reconstitute any images that were transferred, and write the images to disk. Open __recapper.py__ and enter the following code:

In [ ]:
from scapy.all import TCP, rdpcap
from collections import namedtuple
import os
import re
import sys
import zlib

OUTDIR = "/root/Desktop/pictures" #[1]
PCAPS = "/root/Downloads"

Response = namedtuple("Response", ["header", "payload"]) #[2]

def get_header(payload): #[3]
    pass

def extract_content(Response, content_name="image"): #[4]
    pass

class Recapper:
    def __init__(self, fname):
        pass

    def get_responses(self): #[5]
        pass

    def write(self, content_name): #[6]
        pass

if __name__ == "__main__":
    pfile = os.path.join(PCAPS, "pcap.pcap")
    recapper = Recapper(pfile)
    recapper.get_responses()
    recapper.write("image")

This is the main skeleton logic of the entire script, and we'll add in the supporting functions shortly. We set up the imports and then specify the location of the directory in which to output the images and the location of the pcap file to read __[1]__. Then we define a __namedtuple__ called __Response__ to have two attributes: The packet __header__ and packet __payload__ __[2]__. We'll create two helper functions to get the packet header __[3]__ and extract the contents __[4]__ that we'll use with the __Recapper__ class we'll define to reconstitute the images present in the packet stream. Besides __\_\_init\_\___, the __Recapper__ class will have two methods: __get_responses__, which will read responses from the pcap file __[5]__, and __write__, which will write image files contained in the responses to the output directory __[6]__.
Let's start filling out this script by writing the __get_header__ function:

In [ ]:
def get_header(payload):
    try:
        header_raw = payload[:payload.index(b"\r\n\r\n") + 2] #[1]
    except ValueError:
        sys.stdout.write('-')
        sys.stdout.flush()
        return None #[2]

    # The regex is fucked up and the tags are interpreted as html, if you're
    # reading from pdf consider this, fuck you colab.
    header = dict(re.findall(r"(?P<name>.*?): (?P<value>.*?)\r\n", header_raw.decode())) #[3]
    if "Content-Type" not in header: #[4]
        return None
    return header

The __get_header__ function takes the raw HTTP traffic and spits out the headers. We extract the header by looking for the portion of the payload that starts at the beginning and ends with a couple of carriage return and newline pairs __[1]__. If the payload doesn't match that pattern, we'll get a __ValueError__, in which case we just write a dash ('-') to the console and return __[2]__.

Otherwise, we create a dictionary (__header__) from the decoded payload, splitting on the colon so that the key is the part before the colon and the value is the part after the colon __[3]__. If the header has no key called __Content-Type__, we return __None__ to indicate that the header doesn't contain the data we want to extract __[4]__. Now let's write a function to extract the content from the response:

In [ ]:
def extract_content(Response, content_name="image"):
    content, content_type = None, None
    if content_name in Response.header["Content-Type"]: #[1]
        content_type = Response.header["Content-Type"].split('/')[1] #[2]
        content = Response.payload[Response.payload.index(b"\r\n\r\n") + 4:] #[3]

        if "Content-Encoding" in Response.header: #[4]
            if Response.header["Content-Encoding"] == "gzip":
                content = zlib.decompress(Response.payload, zlib.MAX_WBITS | 32)
            elif Response.header["Content-Encoding"] == "deflate":
                content = zlib.decompress(Response.payload)

    return content, content_type #[5]

The __extract_content__ function takes the HTTP response and the name for the content type we want to extract. Recall that __Response__ is a __namedtuple__ with two parts: The header and the payload.

If the content has been encoded __[4]__ with a tool like __gzip__ or __deflate__, we decompress the content by using the __zlib__ module. For any reponse that contains an image, the header will have the name __image__ in the __Content-Type__ attribute (for example, __image/png__ or __image/jpg__) __[1]__. When that occurs, we create a variable named __content_type__ with the actual content type specified in the header __[2]__. We create another variable to hold the content itself, which is everything in the payload after the header __[3]__. Finally, we return a tuple of the __content__ and __content_type__ __[5]__.

With those two helper functions complete, let's fill out the __Recapper__ methods:

In [ ]:
class Recapper:
    def __init__(self, fname): #[1]
        pcap = rdpcap(fname)
        self.sessions = pcap.sessions() #[2]
        self.responses = list() #[3]

First, we initialize the object with the name of the pcap file we want to read __[1]__. We take advantage of a beautiful feature of Scapy to automatically serparate each TCP session __[2]__ into a dictionary that contains each complete TCP stream. Finally, we create an empty list called __responses__ that we're about to fill in with the responses from the pcacp file __[3]__.

In the __get_responses__ method, we will traverse the packets to find each separate __Response__ and add each one to the list of responses present in the packet stream:

In [ ]:
class Recapper:
    def get_responses(self):
        for session in self.sessions: #[1]
            payload = b''
            for packet in self.sessions[session]: #[2]
                try:
                    if packet[TCP].dport == 80 or packet[TCP].sport == 80: #[3]
                        payload += bytes(packet[TCP].payload)
                except IndexError:
                    sys.stdout.write('x') #[4]
                    sys.stdout.flush()
            if payload:
                header = get_header(payload) #[5]
                if header is None: continue
                self.responses.append(Response(header=header, payload=payload)) #[6]

In the __get_responses__ method, we iterate over the __sessions__ dictionary __[1]__, then over the packets within each session __[2]__. We filter the traffic so we get only packets with a destination or source port of 80 __[3]__. Then we concatenate the payload of all the traffic into a single buffer called __payload__. This is effectively the same as right-clicking a packet in Wireshark and selecting _Follow TCP Stream_. If we don't succeed in appending to the payload variable (most likely because there is no TCP in the packet), we print an __x__ to the console and keep going __[4]__.

Then, after we've reassembled the HTTP data, if the __payload__ byte string is not empty, we pass it off to the HTTP header-parsing function __get_header__ __[5]__, which enables us to inspect the HTTP headers individually. Next, we append the __Response__ to the __responses__ list __[6]__.

Finally, we go through the list of responses and, if the response contains an image, we write the image to disk with the __write__ method:

In [ ]:
class Recapper:
    def write(self, content_name):
        for i, response in enumerate(self.responses): #[1]
            content, content_type = extract_content(response, content_name) #[2]
            if content and content_type:
                fname = os.path.join(OUTDIR, f"ex_{i}.{content_type}")
                print(f"Writing {fname}")
                with open(fname, "wb") as f:
                    f.write(content) #[3]

With the extraction work complete, the __write__ method has only to iterate over the responses __[1]__, extract the content __[2]__, and write that content to a file __[3]__. The file is created in the output directory with the names formed by the counter from the __enumerate__ built-in function and the __content_type__ value.

For example, a resulting image name might be _ex_2.jpg_. When we run the program, we create a __Recapper__ object, call its __get_responses__ method to find all the responses in the pcap file, and then write the extracted images from those reponses to disk.

In the next program, we'll examine each image to determine whether it cointans a human face. For each image that has a face, we'll write a new image to disk, adding a box around the face in the image. Open up a new file named __detector.py__:

In [ ]:
import cv2
import os

ROOT = "/rootDesktop/pictures"
FACES = "/root/Desktop/faces"
TRAIN = "/root/Desktop/training"

def detect(srcdir=ROOT, tgtdir=FACES, train_dir=TRAIN):
    for fname in os.listdir(srcdir):
        if not fname.upper().endswith(".JPG"): #[1]
            continue
        fullname = os.path.join(srcdir, fname)
        newname = os.path.join(tgtdir, fname)
        img = cv2.imread(fullname) #[2]
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        training = os.path.join(train_dir, "haarcascade_frontalface_alt.xml")
        cascade = cv2.CascadeClassifier(training) #[3]
        rects = cascade.detectMultiScale(gray, 1.3, 5)
        try:
            if rects.any(): #[4]
                print("Got a face")
                rects[:, 2:] += rects[:, :2] #[5]
        except AttributeError:
            print(f"No faces found in {fname}.")
            continue

        # Highlight the faces in the image
        for x1, y1, x2, y2 in rects:
            cv2.rectangle(img, (x1, y1), (x2, y2), (127, 255, 0), 2) #[6]
        cv2.imwrite(newname, img) #[7]

if __name__ == "__main__":
    detect()

The __detect__ function receives the source directory, the target directory, and the training directory as input. It iterates over the JPG files in the source directory. Since we're looking for faces, the images are presumably photographs, so they're most likely saved as _.jpg_ files __[1]__. We then read the image by using the __OpenCV__ computer vision library __cv2__ __[2]__, load the __detector__ xml file, and create the cv2 face detector object __[3]__. This detector is a classifier that is trained in advance to detect faces in a front-facing orientation. OpenCV contains classifiers for profile (sideways) face detection, hands, fruit, and a whole host of other objects that you can try out for yourself. For images in which faces are found __[4]__, the classifier will return the coordinates of a rectangle that corresponds to where the face was detected in the image. In that case, we print a message to the console, draw a green box around the face __[6]__, and write the image to the output directory __[7]__.

The __rects__ data returned from the detector are of the form (_x_, _y_, _width_, _height_), where _x_, _y_ values provide the coordinates of the lower-left corner of the rectangle, and _width_, _height_ values correspond to the width and height of the rectangle.

We use Python slice syntax __[5]__ to convert from one form to another. That is, we convert the returned __rects__ data to actual coordinates: (_x1_, _y1_, _x1+width_, _y2+height_) or (_x1_, _y1_, _x2_, _y2_). This is the input format the __cv2.rectangle__ method is expecting.

This code was generously shared by Chris Fidao at http://www.fideloper.com/facial-detection/. This example made slight modifications to the original. Now let's take this all for a spin inside your Kali VM.

# *__Kicking the Tires__*

If you haven't first installed the OpenCV libraries, run the following commands (again, fuck you, Chris Figao) from a terminal in your Kali VM:
```
#:> apt-get install libopencv-dev python3-opencv python3-numpy python3-scipy
```
This should install all of the necessary files needed to handle facial detection on the resulting images. We also need to grab the facial detection training file, like so:
```
#:> wget http://eclecti.cc/files/2008/03/haarcascade_frontalface_alt.xml
```
Copy the downloaded file to the directory we specified in the _TRAIN_ variable in __detector.py__. Now create a couple of directories for the output, drop in a pcap, and run the scripts. This should look something like the following:
```
#:> mkdir /root/Desktop/pictures
#:> mkdir /root/Desktop/faces
#:> python recapper.py
Extracted: 189 images
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx--------------xx
Writing pictures/ex_2.gif
Writing pictures/ex_8.jpeg
Writing pictures/ex_9.jpeg
Writing pictures/ex_15.png
...
#:> python detector.py
Got a face
Got a face
...
#:>
```
You might see a number of error messages being produced by OpenCV because some of the images we fed into it may be corrupt or partially downloaded or their format might not be supported. We'll leave building a robust image extraction and validation routine as a homework assignment for you (__PARACULI!!__). If you crack open your _faces_ directory, you should see several files with faces and magic green boxes drawn around them.

This technique can be used to determine what types of content your target is looking at, as well as to discover likely approaches via social engineering. You can, of course, extend this example beyond using it against carved images from pcaps and use it in conjunction with web crawling and parsing techniques decribed in later chapters.